In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [3]:
# !apt-get update # to update ubuntu to correctly run apt install
# !pip install fuzzywuzzy
# !pip install wikipedia

In [78]:
from tqdm.notebook import tqdm_notebook
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np
import wikipedia
import requests
import json
import time
import re
import os

In [91]:
with open("./company_link.json","r+") as link:
    company_links = json.load(link)
print(len(company_links))

100


In [92]:
def return_correctname(company_name,results):
    score_list = []
    max_score = 0
    indx=0
    for k in range(len(results)):
        score = fuzz.ratio(results[k].lower(),company_name)
        # print(f'{results[k]} - {score}')
        if(score>max_score):
            max_score=score 
            indx=k
        score_list.append(score)
    if(max_score<=30): return ''
    else: return results[indx]

def basic_work(title,company_temp_dict):
    # print(title)
    # page_url = wikipedia.WikipediaPage(title=title).url
    temp_url = re.sub(' ','_',title)
    page_url = 'https://en.wikipedia.org/wiki/' + temp_url
    # print(page_url)
    page = requests.get(page_url).text
    soup = BeautifulSoup(page, 'html.parser')
    infobox = soup.find('table',{'class':"infobox vcard"})
    company_temp_dict['wikipedia page url']=wikipedia.WikipediaPage(title=title).url
    if infobox is not None:
        try: 
            ret_table_dict(infobox,company_temp_dict,("table_"+str(i)))
        except Exception as e: 
            print(e)

def ret_parent_table(data):
    # data -> table (tags)
    # Return Value -> String
    rows = data.findChildren('tr')
    table = ""
    i=0
    while (i < len(rows)):
        if(rows[i].findChildren('table')):
            val = len((str(rows[i])).split('<tr>'))-1
            i=i+val
        else:
            table+=str(rows[i])
        i+=1
    table = '<table>' + table + '</table>'
    # print("YO")
    table = str(table)
    table = table.replace("<ul>"," ")
    table = table.replace("</ul>"," ")
    table = table.replace("<li>"," ")
    table = table.replace("</li>",",")
    table = table.replace("<br/>",",")
    return table

def ret_table_dict(data,d_company,val):
    # data -> string format
    # return -> dictionary
    try: 
        table=ret_parent_table(data)
        df = pd.read_html(table)
        table_columns = list(df[0].columns)
        data_list = np.array(df).tolist()[0]
    except Exception as e:
        print(e)
        html_part = re.sub(r'<.*?>', lambda g: g.group(0).upper(), ret_parent_table(data))
        df = pd.read_html(html_part)[0]
        table_columns = list(df.columns)
        data_list = np.array(df).tolist()
    used = []
    for i in table_columns:
        if(len(str(i))>25): return
        if(len(str(i))<0 or len(str(i))=='nan' or str(i).isnumeric()): used.append(False)
        else: used.append(True)
    temp_list = []
    for i in range(len(data_list)):
        if(not used[0] and len(used)==2):
            if(str(data_list[i][0])!='nan' and str(data_list[i][1])!='nan' and len(str(data_list[i][0]))>0 and len(str(data_list[i][1]))>0 and str(data_list[i][0]) != str(data_list[i][1])):
                first_val = re.sub("\[[0-9]+\]", '', data_list[i][0]).strip().lower()
                second_val = re.sub("\[[0-9]+\]", '', data_list[i][1]).strip().lower()
                d_company[first_val]=second_val 
        else:
            dictio = {}
            for j in range(len(table_columns)):
                if(str(data_list[i][j])=='nan' or len(str(data_list[i][j]))<=0): continue
                if(not str(table_columns[j]).isnumeric()): first_val = re.sub("\[[0-9]+\]", '', data_list[i][j]).strip().lower()
                else: first_val = int(table_columns[j])
                second_val = re.sub("\[[0-9]+\]", '', data_list[i][j]).strip().lower()
                dictio[first_val]=second_val
            temp_list.append(dictio)
    if(len(temp_list)>0):
        d_company[val]=temp_list

In [93]:
count = 0
limit = 140000
temp_dict = {}
findings_dict = {}
list_remove = ['private','limited','limit','ed','limite','d','llp','company','pvt','ltd','lim','limi','ted','(opc)','(india)']
with tqdm_notebook(total=len(company_links)) as pbar:
    for i in company_links:
        try:
            company_name = company_links[i]['Company'].lower()
            brokn = company_name.split(' ')
            for j in range(len(brokn)):
                if(brokn[j]in list_remove):
                    brokn[j]=''
            brokn = list(filter(lambda x:len(x)!=0,brokn))
            company_name = ' '.join(brokn)
            results = wikipedia.search(company_links[i]['Company'])
            final_res = return_correctname(company_name,results)
            if(len(final_res)>0):
                company_temp_dict = {
                    'name':company_links[i]['Company'],
                    'wikipedia page':final_res
                }
                basic_work(final_res,company_temp_dict)
                if 'headquarters' in company_temp_dict:
                    val = re.sub(",","",company_temp_dict['headquarters'].lower())
                    temp_val = list(filter(lambda x:len(x)>0,val.split(" ")))
                    if ('hyderabad' in temp_val) or ('telangana' in temp_val):
                        temp_dict[i] = company_temp_dict
            # if(count > limit):
            #     break
            count+=1
        except Exception as e:
            print(e)
        pbar.update()

  0%|          | 0/100 [00:00<?, ?it/s]

In [94]:
print(len(temp_dict))
file = open("./check_result.json", "w+")
json.dump(temp_dict, file, indent=4)
file.close()

31
